# Check mining polygons

In [ ]:
import geopandas as gpd
import shapely
import leafmap.foliumap as leafmap
import rasterio
from shapely.geometry import box
import pandas as pd

# # optional change working directiory first
import os
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

import sys
sys.path.append('..')

from src.data.get_satellite_images import ReadSTAC

In [ ]:
# download mining polygons
import os
parend_dir = os.path.dirname(os.getcwd())
script_path = os.path.join(parend_dir, 'src', 'data', 'get_mining_polygons.py')
os.system(f'python {script_path}')

In [ ]:
MAUS_POLYGONS = "data/external/maus_mining_polygons.gpkg"
MAUS_AREA_RASTER = "data/external/maus_mining_raster.tif"
TANG_POLYGONS = "data/external/tang_mining_polygons/74548_mine_polygons/74548_projected.shp"

# filter both dataframes to only the area of interest
LOCATION = [-50.16556135114535, -6.060451692157381]
BBOX = [LOCATION[0] - 0.5, LOCATION[1] - 0.5, LOCATION[0] + 0.5, LOCATION[1] + 0.5]


In [ ]:
# Load a GeoPackage file into a GeoDataFrame
maus_gdf = gpd.read_file(MAUS_POLYGONS)

# Load a Shapefile into a GeoDataFrame
tang_gdf = gpd.read_file(TANG_POLYGONS)

In [ ]:
maus_gdf.head()

In [ ]:
tang_gdf.head()

In [ ]:
print(maus_gdf.shape)
print(tang_gdf.shape)

In [ ]:
# check how many empty geometries are in the tang_gdf
empty_geometries = tang_gdf[tang_gdf.geometry == None]
len(empty_geometries)

In [ ]:
# convert projected coordinates in tang to lat long
tang_gdf = tang_gdf.to_crs(epsg=4326)

In [ ]:
# Flatten polygon dimensions from 3D to 2D
tang_gdf.geometry = shapely.wkb.loads(shapely.wkb.dumps(tang_gdf.geometry, output_dimension=2))

In [ ]:
maus_gdf_filtered = maus_gdf.cx[BBOX[0]:BBOX[2], BBOX[1]:BBOX[3]]
tang_gdf_filtered = tang_gdf.cx[BBOX[0]:BBOX[2], BBOX[1]:BBOX[3]]

In [ ]:
maus_gdf_filtered.iloc[0,:].geometry.bounds

In [ ]:
maus_gdf_filtered.head()

In [ ]:
tang_gdf_filtered.head()

In [ ]:
# Create a Leaflet map
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)
m.add_basemap("satellite")


style = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
    "fillOpacity": 0.1,
}

# Add the GeoDataFrame to the map
m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")
m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)

# Display the map
m

# Overlay a Sentinel Image with the mining polygons

In [ ]:
# Download the image 
# Option 1 (Default): Read from Planetary Computer STAC API
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02']
bands_landsat = ['red', 'green', 'blue']

stac_reader = ReadSTAC(api_url=api_url, collection = "sentinel-2-l2a")

# check available items
items = stac_reader.get_items(
    location = (-59.66666666666667, 7.33333333333334 ),
    buffer=10,
    timerange='2020-01-01/2020-12-30',
    max_cloud_cover=10
)

## Start with only displaying a small Area of Interest

In [ ]:
stack = stac_reader.get_stack(items, filter_by="least_cloudy", bands=bands, resolution=10)
stack_stretched = stac_reader.stretch_contrast_stack(stack, upper_percentile=1.0, lower_percentile=0.0)
image = stac_reader.save_stack_as_geotiff(stack_stretched, filename="sentinel_image.tif")

In [ ]:
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)

m.add_raster(image, layer_name="Image")
m

In [ ]:
# add the polygons in the area
m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")
m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)
m

## Display the whole S2 Tile

In [ ]:
least_cloudy_item = stac_reader.filter_item(items, filter_by="least_cloudy")

In [ ]:
m = stac_reader.preview_tile_outlines(least_cloudy_item)
m

## Test global MGRS 10x10 km grid and overlay it with mining polygons

In [ ]:
# read the mgrs shapefile
mgrs_gdf = gpd.read_file("data/external/mgrs_index_ftp_link/mgrs_index_ftp_link.shp")

In [ ]:
from shapely.geometry import Polygon
import pyproj
from shapely.ops import transform
from functools import partial

def calculate_dimensions_km(polygon):
    """
    Calculate the dimensions (width, height) in kilometers of a given polygon.
    
    Parameters:
    - polygon: A shapely Polygon object.
    
    Returns:
    - A tuple (width_km, height_km) representing the dimensions in kilometers.
    """
    # Define the projection to UTM (Universal Transverse Mercator)
    # Find UTM zone for the centroid of the polygon for more accuracy
    utm_zone = int((polygon.centroid.x + 180) / 6) + 1
    crs_proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84', preserve_units=False)
    
    # Define transformations from WGS84 to UTM and back
    project_to_utm = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'), crs_proj)
    project_to_wgs84 = partial(pyproj.transform, crs_proj, pyproj.Proj(init='epsg:4326'))
    
    # Transform the polygon to the UTM projection
    polygon_utm = transform(project_to_utm, polygon)
    
    # Calculate bounds in UTM
    minx, miny, maxx, maxy = polygon_utm.bounds
    
    # Calculate width and height in meters
    width_m = maxx - minx
    height_m = maxy - miny
    
    # Convert meters to kilometers
    width_km = width_m / 1000
    height_km = height_m / 1000
    
    return (width_km, height_km)

# Example usage
polygon = mgrs_gdf.iloc[0].geometry
width_km, height_km = calculate_dimensions_km(polygon)
print(f"Width: {width_km} km, Height: {height_km} km")

In [ ]:
mgrs_gdf.iloc[0:2,:].geometry

# add that on a map
m = leafmap.Map()
m.add_gdf(mgrs_gdf.iloc[0:2,:], layer_name="mgrs_gdf")
m

# Load Raster for mining areas from Maus et al

In [ ]:
# Load the raster for mining areas 
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)
m.add_raster(MAUS_AREA_RASTER, layer_name="Mining Areas", alpha=0.5)

# Display the map
m

In [ ]:
def resample_geotiff(source_path, dest_path, resampling_factor): 
    import rasterio
    from rasterio.enums import Resampling

    with rasterio.open(source_path) as dataset:

        # resample data to target shape using upscale_factor
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * resampling_factor),
                int(dataset.width * resampling_factor)
            ),
            resampling=Resampling.average
        )

        print('Shape before resample:', dataset.shape)
        print('Shape after resample:', data.shape[1:])

        # scale image transform
        dst_transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )

        print('Transform before resample:\n', dataset.transform, '\n')
        print('Transform after resample:\n', dst_transform)

        # Write outputs
        # set properties for output
        dst_kwargs = dataset.meta.copy()
        dst_kwargs.update(
            {
                "crs": dataset.crs,
                "transform": dst_transform,
                "width": data.shape[-1],
                "height": data.shape[-2],
                "nodata": 0,  
            }
        )

        with rasterio.open(dest_path, "w", **dst_kwargs) as dst:
            # iterate through bands
            for i in range(data.shape[0]):
                dst.write(data[i].astype(rasterio.uint32), i+1)

In [ ]:
from matplotlib import pyplot
import rasterio
array = rasterio.open(MAUS_AREA_RASTER).read(1)

# set everything smaller than 0 to 0
array = array.clip(0)

pyplot.imshow(array, cmap='pink')
pyplot.show()

In [ ]:
import numpy as np
from tqdm import tqdm
src = rasterio.open("test.tif")
array = src.read(1)

# Get the transformation matrix
transform = src.transform

# Create an empty list to store the bounding boxes
bounding_boxes = []
mining_area = []

# Iterate over the pixels in the raster
# only record bounding box if they have over 0.5 square km of mining area (out of a total area per square of 78.41 sq.km)
for x in tqdm(range(src.width)):
    for y in range(src.height):
        if array[y, x] > 0.5:
            # Get the pixel's bounding box
            # The bounding box is defined by the pixel's top-left and bottom-right corners
            top_left = transform * (x, y)
            bottom_right = transform * (x + 1, y + 1)
            bounding_box = [top_left[0], bottom_right[1], bottom_right[0], top_left[1]]
            
            # Add the bounding box to the list
            bounding_boxes.append(bounding_box)

            # add the mining area to the list
            mining_area.append(array[y, x])


In [ ]:
len(bounding_boxes)

In [ ]:
# Create a GeoDataFrame from the bounding boxes and the area
gdf = gpd.GeoDataFrame(geometry=[box(*bbox) for bbox in bounding_boxes], crs="EPSG:4326")
gdf["mining_area"] = mining_area
gdf

In [ ]:
# Create a Map
m = leafmap.Map(center=[(bounding_boxes[0][1] + bounding_boxes[0][3]) / 2, 
                        (bounding_boxes[0][0] + bounding_boxes[0][2]) / 2], zoom=2)

# Add the GeoDataFrame to the map
m.add_gdf(gdf, layer_name="bounding_boxes")

# Display the map
m

In [ ]:
# save the bounding boxes as a geopackage file
gdf.to_file("/workspaces/mine-segmentation/data/interim/mining_areas.gpkg", driver="GPKG")

# Sample random mining tile, plot 2019 sentinel image, and add both mining datasets to it. 

In [ ]:
# read gdf
gdf = gpd.read_file("/workspaces/mine-segmentation/data/interim/mining_areas.gpkg")

In [ ]:
import random

# Set the random seed for reproducibility
random.seed(1234)

In [ ]:

YEAR = 2019

# Sample a random mining tile
random_tile = gdf.sample(n=1, random_state=random.randint(0, 100))

# Get the geometry of the random tile
tile_geometry = random_tile['geometry'].values[0]

bbox = tile_geometry.bounds

api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02']

stac_reader = ReadSTAC(api_url=api_url, collection = "sentinel-2-l2a")

# get the least cloudy sentinel image for the tile
items = stac_reader.get_items(
    bbox=bbox,
    timerange=f'{YEAR}-01-01/{YEAR}-12-31',
    max_cloud_cover=10
)

stack = stac_reader.get_stack(items, filter_by="least_cloudy", bands=bands, resolution=10)
stack_stretched = stac_reader.stretch_contrast_stack(stack, upper_percentile=0.99, lower_percentile=0.01)
image = stac_reader.save_stack_as_geotiff(stack_stretched, filename="sentinel_image.tif")

# Create a Map
m = leafmap.Map(center=[tile_geometry.centroid.y, tile_geometry.centroid.x], zoom=2)

# add the image
m.add_raster(image)

# Filter the polygons that are included 
maus_gdf_filtered = maus_gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
tang_gdf_filtered = tang_gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

style = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
    "fillOpacity": 0.1,
}

# Display the filtered gdfs
if not maus_gdf_filtered.empty:
    m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")

if not tang_gdf_filtered.empty:
    m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)

m

In [ ]:
# load mining areas
mining_areas = gpd.read_file("/workspaces/mine-segmentation/data/interim/mining_areas.gpkg")

# get the tile number 4,242
tile = mining_areas.iloc[4242:4244, :]

# plot that tile on a map
m = leafmap.Map(zoom=10)
m.add_gdf(tile, layer_name="tile")
m



In [ ]:
print(tile.iloc[0,:].geometry)